<a href="https://colab.research.google.com/github/emily2925/DS-Class-Project-Titanic/blob/main/%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F%E5%B0%88%E6%A1%88%E5%AF%A6%E4%BD%9C_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

目的：找出鐵達尼號事件生存的關鍵因素

# New Section

Packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

Data Reading and Explore (from Base)
1. 整體而言，死:生=5:3
2. class-> 1:2:3 = 2:1:4，多半住3等艙
3. sex->男:女 = 5:3
4. 多半在S港上船
5. 多半親屬朋友家人數為1

Data Reading and Explore - Numeric Data (from Base)
1. Fare 關聯最大，正向(0.25)
2. Age負向、SibSp負向、Parch正向(皆0.0x)
3. 從數字分配來看，其餘差異不大，明顯Live組Fare比較高
4. Fare為關鍵因素

Data Reading and Explore - Categorical Data (from Base)
1. Pclass大，表示艙位越低等，存活率低(-0.3)
2. 女性存活率大於男性(0.5)
3. 小孩存活率高(0.12)
4. 在C港上船存活率高(0.16)，在S存活率低(-0.15)
5. Pclass,Sex為關鍵因素

Model Base (from Base)
1. 發現目前的方法就很準
2. 最高0.8033,最低0.76
3. Feauture:"Survived", "Fare", "Pclass", "female", "C", "S", "Kid_", "Adult_"

In [2]:
# data definition:https://www.kaggle.com/c/titanic/data
df = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [30]:
df_all = df

Data Cleaning

In [6]:
# Age
df_all.corr()

<ipython-input-6-746510265ea9>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_all.corr()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [31]:
df_age_mean = df_all[["Pclass", "Age", "SibSp"]].groupby(["Pclass", "SibSp"]).mean().reset_index()

In [32]:
df_age_mean

,Pclass,SibSp,Age
0,1,0,39.181416
1,1,1,37.414154
2,1,2,37.200000
3,1,3,22.000000
4,2,0,31.934220
5,2,1,27.363636
6,2,2,19.125000
7,2,3,30.000000
8,3,0,27.630201
9,3,1,24.912698


In [35]:
df_all["Age"].fillna(10.2, inplace=True)

In [37]:
labels = ["Kid_", "Adult_", "Elder_"]
bins = [0, 13, 50, 110]
df_all["age_group_"] = pd.cut(df_all["Age"], labels=labels, bins=bins)

In [38]:
# Encoding

In [39]:
one_hot = OneHotEncoder()
encoded = one_hot.fit_transform(df_all[["Sex"]])
df_all[one_hot.categories_[0]] = encoded.toarray()

In [40]:
one_hot = OneHotEncoder()
encoded = one_hot.fit_transform(df_all[["Embarked"]])
df_all[one_hot.categories_[0]] = encoded.toarray()

In [41]:
one_hot = OneHotEncoder()
encoded = one_hot.fit_transform(df_all[["age_group_"]])
df_all[one_hot.categories_[0]] = encoded.toarray()

In [43]:
# Embarked
df_all.corr()

<ipython-input-43-75d5ed28726d>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_all.corr()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S,NaN,Adult_,Elder_,Kid_
PassengerId,1.000000e+00,-0.005007,-0.035144,0.038850,-0.057527,-0.001652,0.012658,-0.042939,0.042939,-0.001205,-0.033606,0.022148,-6.756184e-19,0.023600,-0.002298,-0.023795
Survived,-5.006661e-03,1.000000,-0.338481,-0.012536,-0.035322,0.081629,0.257307,0.543351,-0.543351,0.168240,0.003650,-0.155660,6.009485e-02,0.018179,-0.022932,-0.006138
Pclass,-3.514399e-02,-0.338481,1.000000,-0.373115,0.083081,0.018443,-0.549500,-0.131900,0.131900,-0.243292,0.221009,0.081720,-7.428233e-02,-0.072374,-0.253606,0.223138
Age,3.884993e-02,-0.012536,-0.373115,1.000000,-0.209327,-0.089995,0.130293,-0.043840,0.043840,0.010387,-0.185208,0.099178,7.570666e-02,0.332283,0.604167,-0.701738
SibSp,-5.752683e-02,-0.035322,0.083081,-0.209327,1.000000,0.414838,0.159651,0.114631,-0.114631,-0.059528,-0.026354,0.070941,-2.250825e-02,-0.174671,-0.068910,0.225612
Parch,-1.652012e-03,0.081629,0.018443,-0.089995,0.414838,1.000000,0.216225,0.245489,-0.245489,-0.011069,-0.081228,0.063036,-2.246691e-02,-0.087449,-0.029255,0.109930
Fare,1.265822e-02,0.257307,-0.549500,0.130293,0.159651,0.216225,1.000000,0.182333,-0.182333,0.269335,-0.117216,-0.166603,4.564557e-02,0.053635,0.063857,-0.093876
female,-4.293888e-02,0.543351,-0.131900,-0.043840,0.114631,0.245489,0.182333,1.000000,-1.000000,0.082853,0.074115,-0.125722,6.429648e-02,0.029318,-0.050538,-0.002089
male,4.293888e-02,-0.543351,0.131900,0.043840,-0.114631,-0.245489,-0.182333,-1.000000,1.000000,-0.082853,-0.074115,0.125722,-6.429648e-02,-0.029318,0.050538,0.002089
C,-1.204795e-03,0.168240,-0.243292,0.010387,-0.059528,-0.011069,0.269335,0.082853,-0.082853,1.000000,-0.148258,-0.778359,-2.286386e-02,-0.037123,0.032589,0.020737


In [44]:
df_all[df_all["Embarked"].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,age_group_,female,male,C,Q,S,NaN,Adult_,Elder_,Kid_
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,...,Adult_,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,...,Elder_,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [47]:
df_all[["Embarked", "Fare"]].groupby("Embarked").mean()

,Fare
Embarked,
C,59.954144
Q,13.276030
S,27.079812


In [49]:
df_all["Embarked"].fillna("C", inplace=True)

In [52]:
df_all.drop(columns=["C", "Q", "S"], inplace=True)

In [53]:
df_all = df_all.loc[:, df_all.columns.notna()]

In [55]:
one_hot = OneHotEncoder()
encoded = one_hot.fit_transform(df_all[["Embarked"]])
df_all[one_hot.categories_[0]] = encoded.toarray()

In [58]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    int64   
 2   Pclass       891 non-null    int64   
 3   Name         891 non-null    object  
 4   Sex          891 non-null    object  
 5   Age          891 non-null    float64 
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int64   
 8   Ticket       891 non-null    object  
 9   Fare         891 non-null    float64 
 10  Cabin        204 non-null    object  
 11  Embarked     891 non-null    object  
 12  age_group_   891 non-null    category
 13  female       891 non-null    float64 
 14  male         891 non-null    float64 
 15  Adult_       891 non-null    float64 
 16  Elder_       891 non-null    float64 
 17  Kid_         891 non-null    float64 
 18  C            891 non-null    f

In [ ]:
# Feature Engineering

In [93]:
df_all.corr()

<ipython-input-93-642ccc8616eb>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_all.corr()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,Adult_,Elder_,Kid_,C,Q,S,Embarked_label,asset_level,Age_group_2,family_number
PassengerId,1.000000,-0.005007,-0.035144,0.038850,-0.057527,-0.001652,0.012658,-0.042939,0.042939,0.023600,-0.002298,-0.023795,-0.001199,-0.033606,0.022148,-0.013078,0.005624,-0.023600,-0.040143
Survived,-0.005007,1.000000,-0.338481,-0.012536,-0.035322,0.081629,0.257307,0.543351,-0.543351,0.018179,-0.022932,-0.006138,0.174718,0.003650,-0.155660,0.174199,0.211747,-0.018179,0.016639
Pclass,-0.035144,-0.338481,1.000000,-0.373115,0.083081,0.018443,-0.549500,-0.131900,0.131900,-0.072374,-0.253606,0.223138,-0.251139,0.221009,0.081720,-0.170334,-0.337348,0.072374,0.065997
Age,0.038850,-0.012536,-0.373115,1.000000,-0.209327,-0.089995,0.130293,-0.043840,0.043840,0.332283,0.604167,-0.701738,0.019458,-0.185208,0.099178,-0.046278,0.034758,-0.332283,-0.188027
SibSp,-0.057527,-0.035322,0.083081,-0.209327,1.000000,0.414838,0.159651,0.114631,-0.114631,-0.174671,-0.068910,0.225612,-0.061970,-0.026354,0.070941,-0.070653,0.165456,0.174671,0.890712
Parch,-0.001652,0.081629,0.018443,-0.089995,0.414838,1.000000,0.216225,0.245489,-0.245489,-0.087449,-0.029255,0.109930,-0.013725,-0.081228,0.063036,-0.042325,0.185083,0.087449,0.783111
Fare,0.012658,0.257307,-0.549500,0.130293,0.159651,0.216225,1.000000,0.182333,-0.182333,0.053635,0.063857,-0.093876,0.273614,-0.117216,-0.166603,0.229304,0.871616,-0.053635,0.217138
female,-0.042939,0.543351,-0.131900,-0.043840,0.114631,0.245489,0.182333,1.000000,-1.000000,0.029318,-0.050538,-0.002089,0.090223,0.074115,-0.125722,0.115513,0.161560,-0.029318,0.200988
male,0.042939,-0.543351,0.131900,0.043840,-0.114631,-0.245489,-0.182333,-1.000000,1.000000,-0.029318,0.050538,0.002089,-0.090223,-0.074115,0.125722,-0.115513,-0.161560,0.029318,-0.200988
Adult_,0.023600,0.018179,-0.072374,0.332283,-0.174671,-0.087449,0.053635,0.029318,-0.029318,1.000000,-0.378965,-0.846023,-0.038749,-0.243140,0.186638,-0.124382,0.005756,-1.000000,-0.163069


In [74]:
# Pclas, Fare, Embarked有某種關聯性，其中，Embarked應該改用有序label，因為C存活率最高，再來Q，再來S

In [75]:
df_all["Embarked_label"] = df_all["Embarked"].apply(lambda x: ["S", "Q", "C"].index(x)+1)

In [77]:
df_all["asset_level"] = df_all["Pclass"] * df_all["Embarked_label"] * df_all["Fare"]

In [ ]:
# Age。因爲已知老人與小孩存活率高，即便相關性時期不高，可合併kid和elders

In [85]:
df_all["Age_group_2"] = np.where(df_all["Adult_"]!=1, 1, 0)

In [ ]:
# 家人數的合併

In [92]:
df_all["family_number"] = df_all["SibSp"] + df_all["Parch"]

Model V1
1. 填完Embarked和Age缺失值後反而變差(0.7808, 0.7528)
2. 加入家人數欄位變好，但和原本base model差異不大，可能填值的誤差被加入欄位補足(0.8089, 0.7640)
3. 加入財富階層，數字差異不大，但0.8的頻率變高(0.8089, 0.7528)
4. 加入較合併的年齡因子又回到原點，故不加
5. 加入家人數，(0.8033, 0.7640)
6. 只放kid，(0.8089, 0.7640)

-> 2, 6有比base model好，6更精練

In [59]:
df_model = df_all[["Survived", "Fare", "Pclass", "female", "C", "S", "Kid_", "Adult_"]] # 填完缺失值

In [68]:
df_model = df_all[["Survived", "Fare", "Pclass", "female", "C", "S", "Kid_", "Adult_", "SibSp", "Parch"]] # 加入家人數

In [80]:
df_model = df_all[["Survived", "female", "Kid_", "Adult_", "SibSp", "Parch", "asset_level"]] # 加入財富階層feature，刪去因子

In [89]:
# df_model = df_all[["Survived", "female", "Age_group_2", "SibSp", "Parch", "asset_level"]] # 加入年齡feature，刪去因子

In [94]:
df_model = df_all[["Survived", "female", "Kid_", "Adult_", "family_number", "asset_level"]] # 加入家人數

In [97]:
df_model = df_all[["Survived", "female", "Kid_", "family_number", "asset_level"]] # 嘗試只放kid

In [98]:
columns_X = set(df_model.columns) - {'Survived'}
columns_y = ['Survived']


train_X = df_model[columns_X]
train_y = df_model[columns_y]

<ipython-input-98-34179671f73f>:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  train_X = df_model[columns_X]


In [99]:
log = LogisticRegression(random_state=0, max_iter=3000)
score = cross_val_score(log, train_X, train_y.values.ravel(), cv=5, scoring="accuracy")
print(score)

[0.79888268 0.80337079 0.76966292 0.76404494 0.80898876]
